In [4]:
from __future__ import print_function
import argparse
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:
from AIToolbox.torchtrain.train_loop import TrainLoop, TrainLoopCheckpointEndSave
from AIToolbox.torchtrain.model import ModelWrap
from AIToolbox.torchtrain.data.batch_model_feed_defs import AbstractModelFeedDefinition
from AIToolbox.torchtrain.callbacks.performance_eval_callbacks import ModelPerformanceEvaluation, ModelPerformancePrintReport
from AIToolbox.experiment.result_package.basic_packages import ClassificationResultPackage

In [ ]:
THIS_DIR = os.path.dirname(os.path.abspath(__file__))

## Define the model

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Define the batch feed def

In [24]:
class MNISTModelFeedDefinition(AbstractModelFeedDefinition):
    def get_loss(self, model, batch_data, criterion, device):
        data, target = batch_data
        data, target = data.to(device), target.to(device)

        output = model(data)
        loss = criterion(output, target)

        return loss
    
    def get_loss_eval(self, model, batch_data, criterion, device):
        return self.get_loss(model, batch_data, criterion, device)

    def get_predictions(self, model, batch_data, device):
        data, y_test = batch_data
        data = data.to(device)

        output = model(data)
        y_pred = output.argmax(dim=1, keepdim=False)  # get the index of the max log-probability

        return y_pred.cpu(), y_test, {}

In [10]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [12]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True, **kwargs)

In [13]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=100, shuffle=True, **kwargs)

In [15]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
criterion = F.nll_loss

In [26]:
callbacks = [ModelPerformanceEvaluation(ClassificationResultPackage(), {'batch_size': 100, 'lr': 0.001},
                                        on_train_data=True, on_val_data=True),
             ModelPerformancePrintReport(['train_Accuracy', 'val_Accuracy'], strict_metric_reporting=True)]

In [ ]:
TrainLoop(ModelWrap(model, MNISTModelFeedDefinition()),
          train_loader, test_loader, None,
          optimizer, criterion)(num_epochs=3, callbacks=callbacks)

## Experiment tracking training

In [28]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
criterion = F.nll_loss

In [29]:
callbacks = [ModelPerformanceEvaluation(ClassificationResultPackage(), {'batch_size': 100, 'lr': 0.001},
                                        on_train_data=True, on_val_data=True),
             ModelPerformancePrintReport(['train_Accuracy', 'val_Accuracy'], strict_metric_reporting=True)]

In [ ]:
TrainLoopCheckpointEndSave(ModelWrap(model, MNISTModelFeedDefinition()),
                                train_loader, test_loader, test_loader,
                                optimizer, criterion,
                                project_name='localRunCNNTest',
                                experiment_name='CNN_MNIST_test',
                                local_model_result_folder_path=THIS_DIR,
                                hyperparams={'batch_size': 100, 'lr': 0.001},
                                test_result_package=ClassificationResultPackage(),
                                cloud_save_mode=None)(num_epochs=3, callbacks=callbacks)